# Phase 01: Collecting data

In [10]:
import requests
import numpy as np
import pandas as pd 
from bs4 import BeautifulSoup
from selenium import webdriver

from selenium.webdriver.chrome.options import Options

import re
# driver = webdriver.Chrome(ChromeDriverManager().install())

### Step 01: Collect apartment links:

##Initials:

I create an empy data frame to store the infomation of the Real estate

In [11]:
fields = [ 'Address', 'Area', 'Price', 'Bedroom', 'Toilet', 'Floor', 'Furniture', 'Direction','Legal', 'Posting date', 'Expiry date', 'Ad type', 'Ad code']

df = pd.DataFrame(columns = fields)

print(df.shape)

(0, 13)


I collect link from many page (each page contain many links to apartments) and store them in a text file.

In [12]:
file_name = "apartment_links.txt"

url = 'https://batdongsan.com.vn/nha-dat-ban-tp-hcm'

First, I get the total num page of the website.

In [14]:
# driver = webdriver.Chrome()
driver = webdriver.Chrome(ChromeDriverManager().install())

driver.get(url)
soup = BeautifulSoup(driver.page_source, 'html.parser')
unprocessed_num_page = soup.find(class_ = 're__pagination-group')
driver.quit()

In [15]:

regex = r'[^\s]+'
matchs = re.findall(regex, unprocessed_num_page.text)
new_array = []
for match in matchs:
    new_array.append(int(match.replace('.', '')))

total_page = max(new_array)
print (total_page)

2500


In [22]:
def CollectApartmentLink (url, page):
    driver = webdriver.Chrome(ChromeDriverManager().install())
    with open (file_name, 'w') as file:

        for i in range(1, page + 1):
            
            # driver = webdriver.Chrome()
            driver.get(url+'/p'+ str(i))
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            apartment_list = soup.find_all("a", class_= "js__product-link-for-product-id")

            for apartment in apartment_list:
                link = 'https://batdongsan.com.vn/'+ apartment['href']
                file.write(link)
                file.write('\n')

            driver.quit()

In [21]:
CollectApartmentLink(url, total_page)

MaxRetryError: HTTPConnectionPool(host='127.0.0.1', port=55772): Max retries exceeded with url: /session/f299555d2d4077aebbb01b5bb104bc88/url (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000002536DC52190>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))

[TEST]Collect links from a group of page

In [23]:
with open (file_name, 'a') as file:
    for i in range(1001, 1100 + 1):
        # driver = webdriver.Chrome()
            # driver = webdriver.Chrome()
        driver = webdriver.Chrome(ChromeDriverManager().install())

        driver.get(url+'/p'+ str(i))
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        apartment_list = soup.find_all("a", class_= "js__product-link-for-product-id")

        for apartment in apartment_list:
            link = 'https://batdongsan.com.vn/'+ apartment['href']
            file.write(link)
            file.write('\n')

        driver.quit()

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=119.0.6045.160)


In [7]:
key_map = {
    'Địa chỉ:' : 'Address',
    'Diện tích': 'Area',
    'Mức giá': 'Price',
    'Hướng nhà': 'Direction',
    'Số tầng': 'Floor',
    'Số phòng ngủ': 'Bedroom',
    'Số toilet': 'Toilet',
    'Pháp lý': 'Legal',
    'Nội thất': 'Furniture',
    'Ngày đăng': 'Posting date',
    'Ngày hết hạn': 'Expiry date',
    'Loại tin': 'Ad type',
    'Mã tin': 'Ad code'
}

Extract data from a link

In [8]:
def ExtractDataFromLink(url):
    driver = webdriver.Chrome()

    try:
        driver.get(url)

        soup = BeautifulSoup(driver.page_source, 'html.parser')

        #Create a dicitonary to store the feature
        features = {}

        # get the address
        address = soup.find(class_ = "re__pr-short-description js__pr-address")
        features['Address'] = address

        # Get all the features
        feature_items = soup.find_all('div', class_ = 're__pr-specs-content-item')
        for item in feature_items:
            title = item.find(class_ = 're__pr-specs-content-item-title').text.strip()
            value = item.find(class_ = 're__pr-specs-content-item-value').text.strip()
            features[title] = value

        date_items = soup.find_all( 'div', class_ = "re__pr-short-info-item js__pr-config-item")
        for item in date_items:
            title = item.find('span', class_ = 'title').text.strip()
            value = item.find('span',class_ = 'value').text.strip()
            features[title] = value
    
        keys_to_change = list(features.keys())

        for key in keys_to_change:
            if key in key_map:
                features[key_map[key]] = features[key]
    finally:
        driver.quit()

    return features

In [ ]:
# fields = [ 'Address', 'Area', 'Price', 'Bedroom', 'Toilet', 'Floor', 'Furniture', 'Direction','Legal', 'Posting date', 'Expiry date', 'Ad type', 'Ad code']

urls = []

with open(file_name, 'r') as file:
        for line in file.readlines():
                urls.append(line)

In [ ]:
for url in urls[: 2]:
        features = ExtractDataFromLink(url)
        new_row = pd.Series(features, index = fields)
        df = pd.concat([df, pd.DataFrame([new_row])], ignore_index = True)


In [9]:
url = 'https://batdongsan.com.vn//ban-can-ho-chung-cu-duong-vo-chi-cong-phuong-phu-huu-2-prj-safira-khang-dien/-chinh-chu-ban-1pn-1-50m2-block-c2-tang-thap-nha-dep-pr38612753'

features = ExtractDataFromLink(url)
new_row = pd.Series(features, index = fields)
df = pd.concat([df, pd.DataFrame([new_row])], ignore_index = True)

In [10]:
df.head()

,Address,Area,Price,Bedroom,Toilet,Floor,Furniture,Direction,Legal,Posting date,Expiry date,Ad type,Ad code
0,"[Dự án Safira Khang Điền, Đường Võ Chí Công, P...",50 m²,"2,27 tỷ",1 phòng,1 phòng,NaN,Đầy đủ,NaN,Sổ đỏ/ Sổ hồng,22/11/2023,29/11/2023,Tin thường,38612753
